>Base Clients : "https://assets-datascientest.s3.eu-west-1.amazonaws.com/mc_intro_de/Clients.csv"
>
>Base Produits : "https://assets-datascientest.s3.eu-west-1.amazonaws.com/mc_intro_de/Products.csv"
>
>Logs : "https://assets-datascientest.s3.eu-west-1.amazonaws.com/mc_intro_de/transactions.logs"
>
>What is the aim of the exercise?
>
> => We want to build an ETL tool using pandas.
>
>  1. We extract data.
>
>  2. We're going to transform it so that we can analyze it. Our analysis will focus on transactions and customers. **For example, we could try to identify the next transaction date by customer: Machine Learning objective)**.
>    - Identify the join key.
>    - Perform the join.
>    - Restructure data, Feature Engineering, etc.
>      - How many days before the current transaction? "DaysUntilNextTransaction"
>      - How many transactions before the current transaction? "NumberOfPastTransactions"
>      - On which day of the week is the transaction carried out? "TransactionWeekDay"
>      - On what date is the transaction carried out? "TransactionDay"
>      - How many transactions per day? "NumberOfTransactionsThisDay"
>    - Keep only the variables that will be used.
>
>  3. Load the dataset

In [ ]:
# library import
import pandas as pd

In [ ]:
# Creation of the following dataframes: df_product, df_client, df_transaction
# For df_transaction : 
# - remember to change the date from object to date
# - column names are: "TransactionID", "Date", "ClientID", "ProductID", "Quantity".

In [ ]:
df_client = pd.read_csv("https://assets-datascientest.s3.eu-west-1.amazonaws.com/mc_intro_de/Clients.csv", index_col=0)

In [ ]:
df_product = pd.read_csv("https://assets-datascientest.s3.eu-west-1.amazonaws.com/mc_intro_de/Products.csv", index_col=0)

In [ ]:
df_transaction = pd.read_csv("https://assets-datascientest.s3.eu-west-1.amazonaws.com/mc_intro_de/transactions.logs",
        sep="---",
        engine='python',
        names=["TransactionID", "Date", "ClientID", "ProductID", "Quantity"],
        parse_dates=["Date"],
        index_col=0,
        nrows=100)
display(df_transaction.head(3), df_transaction.info())

# Extract

In [ ]:
# Creation of an extract() function that creates and returns the 3 dataframes

def extract():
    df_transaction = pd.read_csv("https://assets-datascientest.s3.eu-west-1.amazonaws.com/mc_intro_de/transactions.logs",
            sep='---',
            header=None,
            engine='python',
            names=["TransactionID", "Date", "ClientID", "ProductID", "Quantity"],
            nrows=2000,
            parse_dates=["Date"],
            index_col=0
           )
    df_client = pd.read_csv("https://assets-datascientest.s3.eu-west-1.amazonaws.com/mc_intro_de/Clients.csv", index_col=0)
    df_product = pd.read_csv("https://assets-datascientest.s3.eu-west-1.amazonaws.com/mc_intro_de/Products.csv", index_col=0)
    return df_client, df_product, df_transaction

In [ ]:
# Save dataframes using extract()
df_client, df_product, df_transaction = extract()

In [ ]:
# Use the display() method to get an overview of your dataframes
display(df_client.head(5), df_product.head(5), df_transaction.head(5))

# Transform

In [ ]:
# Join the 3 dataframes together

df = df_transaction.merge(df_client, on="ClientID", how="left").merge(df_product, on="ProductID", how="left")

In [ ]:
df.head()

In [ ]:
# Order lines per client and date

df = df.sort_values(["ClientID", "Date"])

In [ ]:
# Create the DayUntilNextTransactions column : go check pandas.Series.shift() and pandas.Series.dt

In [ ]:
df["DayUntilNextTransactions"] = (df.groupby("ClientID")["Date"].shift(-1) - df["Date"]).dt.days

In [ ]:
# Create the NumberOfPastTransactions column : go check cumcount()
df["NumberOfPastTransactions"] = df.groupby("ClientID").cumcount()

In [ ]:
# Create the TransactionWeekDay column : go check weekday
df["TransactionWeekDay"] = df["Date"].dt.weekday

In [ ]:
# Extract the transaction date without the hour in a TransactionDay column
df["TransactionDay"] = df["Date"].dt.date

In [ ]:
# Create the NumberOfTransactionsPerDay column : go check cumcount()
df["NumberOfTransactionsPerDay"] = df.groupby(["ClientID", "TransactionDay"]).cumcount()

In [ ]:
# Delete the useless columns : ClientName, ClientAddress, ClientCity, ClientZipCode,
# ProductName, ClientState and TransactionDay

df.drop(columns=["ClientName", "ClientAddress", "ClientCity", "ClientZipCode", "ProductName", "ClientState", "TransactionDay"])

In [ ]:
# Delete lines with null values
df = df.dropna()

In [ ]:
# Create a function named transform(df_client, df_product, df_transaction) which transforms the 3 dataframes
# and returns the final dataframe

def transform(df_client, df_product, df_transaction):

    # mergez les 3 dataframes
    df = df_transaction.merge(df_client, on="ClientID", how="left").merge(df_product, on="ProductID", how="left")

    # ordonnez les lignes par Client et par date
    df = df.sort_values(by=["ClientID", "Date"])
    
    # Créez la colonne DayUntilNextTransactions : allez voir pandas.Series.shift() et pandas.Series.dt
    df["DayUntilNextTransactions"] = (df["Date"] - df.groupby("ClientID")["Date"].shift(1)).dt.days

    # Créez la colonne NumberOfPastTransactions : allez voir cumcount()
    df["NumberOfPastTransactions"] = df.groupby("ClientID").cumcount()
    
    # Récuperez le jour de la transaction sans l'heure
    df["TransactionDay"] = df["Date"].dt.date

    # Créez la colonne TransactionWeekDay : allez voir weekday
    df["TransactionWeekDay"] = df["Date"].dt.weekday

    # Créez la colonne NumberOfTransactionsPerDay : allez voir cumcount()
    df["NumberOfTransactionsPerDay"] = df.groupby(["ClientID", "TransactionDay"]).cumcount()

    # Supprimez les colonnes inutiles
    df = df.drop(columns=["ClientName", "ClientAddress", "ClientCity", "ClientZipCode", "ProductName", "ClientState"])

    # Supprimez les lignes avec des valeurs null
    df = df.dropna()

    return df

# Load

In [ ]:
from datetime import datetime

In [ ]:
# Create a load(df) function that saves the dataframe in a file named analyze_<today_date>.csv
# Remember to import the necessary library
def load(df):
    today_str = datetime.strftime(datetime.today(), "%Y-%m-%d")
    filename = "analyse" + today_str + ".csv"
    df.to_csv(filename, index=True, index_label='TransactionID')

In [ ]:
load(transform(*extract()))